# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [38]:
#!pip install selenium
#Config basica para funcionar

# importanto as bibliotecas como a documentação do selenium recomenda
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#path = r'C:\Users\thales.santos\Anaconda3\chromedriver.exe'



In [70]:
navegador = webdriver.Chrome()
navegador.get("https://www.google.com.br")

# Passo 1: Pegar a cotação do dólar
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Dolar")    
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)  
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')             
print(cotacao_dolar)

# Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com.br")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Euro")    
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)  
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')             
print(cotacao_euro)

# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20279%2C47.")
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()


# Passo 5: Exportar a base de preços atualizada

5.2935
5.127839917
284.43


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [57]:
# Passo 4: Atualizar a base de preços (Atualizado o preço de compra e o de venda)
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [72]:
#Atualizar a coluna de cotação
#tabela.loc[tabela["Moeda"] == "Ouro", "Cotação]df.replace({',': '.'}, regex=True)
# Eu quero editar a coluna Cotação, onde a Coluna Moeda = Dólar
tabela.loc[tabela['Moeda'] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == "Ouro", "Cotação"] = float(cotacao_ouro)

#Atualizar a coluna de preço de compra = preço original =  cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar a coluna de preço de venda = preço de compra = margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.29350,5293.447065,1.40,7410.825891
1,Carro Renault,4500.00,Euro,5.12784,23075.279626,2.00,46150.559253
2,Notebook Dell,899.99,Dólar,5.29350,4764.097065,1.70,8098.965011
3,IPhone,799.00,Dólar,5.29350,4229.506500,1.70,7190.161050
4,Carro Fiat,3000.00,Euro,5.12784,15383.519751,1.90,29228.687527
5,Celular Xiaomi,480.48,Dólar,5.29350,2543.420880,2.00,5086.841760
6,Joia 20g,20.00,Ouro,284.43000,5688.600000,1.15,6541.890000


### Agora vamos exportar a nova base de preços atualizada

In [ ]:
#passo 5: Exportar a base de preços atualizado
tabela.to_excel("Produtos Preços Atualizados.xlsx", index=False)